# **Artszy**: The ML Models Gradio Demo:

Using Gradio 3.0, this demo is designed to showcase the backend of the Artszy Platform. The Final Product would be a web app built using React, FastAPI and Python.

This demo showcases everything but the UI of the platform.

-Bharath Raj

##Installing Dependencies:

In [ ]:
from google.colab import output

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: Please use low quality or low res <--"
  print(warning_string)
  output.eval_js('alert("Warning - low GPU (see message)")')
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  !pip install torch==1.9.0+cu102 torchtext==0.10.0 torchvision==0.10.0+cu102 torch-optimizer==0.1.0 -f https://download.pytorch.org/whl/torch/ -f https://download.pytorch.org/whl/torchvision/
  !git clone https://github.com/openai/CLIP
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone https://github.com/mfrashad/clipit.git
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia==0.6.1
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !pip install gradio
  
  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..
  
  !mkdir -p steps
  !mkdir -p models

output.clear()
import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))


GPU check: 15109 MiB available: this should be fine


## Importing Modules for Style Transfer and Super Resolution

In [2]:
import os
import tensorflow as tf
import numpy as np
import PIL.Image
import time
import functools
import tensorflow_hub as hub


##Importing Modules for VQGAN + Clip

In [3]:
import torch
import clipit

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3883008/45929032 bytes (8.5%)7823360/45929032 bytes (17.0%)11812864/45929032 bytes (25.7%)15892480/45929032 bytes (34.6%)20004864/45929032 bytes (43.6%)24109056/45929032 bytes (52.5%)28172288/45929032 bytes (61.3%)32186368/45929032 bytes (70.1%)36102144/45929032 bytes (78.6%)39952384/45929032 bytes (87.0%)43859968/45929032 bytes (95.5%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

##Function Definitions:

#####Style Transfer:

In [4]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)


def convert_img(img, quality):
  max_dim = 256 * quality
  img = tf.image.convert_image_dtype(img, tf.float32)
  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def style_transfer(content, style, quality):
    hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    content = convert_img(content, quality)
    style = convert_img(style, quality)
    stylized_image = hub_model(tf.constant(content), tf.constant(style))[0]
    return tensor_to_image(stylized_image)

#####VQGAN + Clip:

In [12]:
def generate(prompt, generation_quality, detail):
    detail_dict = {'draft':30, 'normal': 80, 'better': 150}
    torch.cuda.empty_cache()
    clipit.reset_settings()
    clipit.add_settings(prompts=prompt,
                        aspect='square',
                        quality=generation_quality,
                        use_pixeldraw=False,
                        use_clipdraw=False,
                        make_video=False,
                        iterations=detail_dict[detail])
    settings = clipit.apply_settings()
    clipit.do_init(settings)
    clipit.do_run(settings)

    return 'output.png'

#Gradio Interface:



In [6]:
%%capture
!pip install gradio

In [8]:
import gradio as gr

##Style Transfer:

In [26]:
%%capture
content = gr.inputs.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="content")
style = gr.inputs.Image(shape=None, image_mode="RGB", invert_colors=False, source="upload", tool="editor", type="numpy", label="Style")
quality = gr.inputs.Radio(choices=[1, 2, 3, 4], label="Quality")

editor = gr.Interface(style_transfer, 
                      inputs=[content, style, quality], 
                      title = "Style Transfer",
                      article = "Style transfer is a computer vision technique"
                      +"that allows us to recompose the content of an image in the style of another",
                      outputs=["image"],
                      allow_flagging = "never",
                      live=False)

  DeprecationWarning,

  warnings.warn(value)

  DeprecationWarning,

  DeprecationWarning,



## VQGAN + CLIP:

In [ ]:
%%capture
prompt = gr.inputs.Textbox(default="Flying Cars", label="Text Prompt")
generation_quality = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Resolution")
detail = gr.inputs.Radio(choices=['draft', 'normal', 'better'], label="Detail")


generator = gr.Interface(generate, 
                         inputs=[prompt, generation_quality, detail], 
                         outputs=['image'], 
                         title = "Text to Art",
                         enable_queue=True, 
                         live=False)

## Combined Interface

In [ ]:
combined = gr.TabbedInterface([generator, editor], ["Generate Art", "Style Transfer"])
combined.launch(debug=True)

Clean Cache:

In [30]:
torch.cuda.empty_cache()